In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
from random import randint

In [ ]:
class vgg_convnet(nn.Module):

    def __init__(self):

        super(vgg_convnet, self).__init__()

        self.conv1a = nn.Conv2d(3,   64,  kernel_size=3,padding=1)
        self.conv1b = nn.Conv2d(64,  64,  kernel_size=3,padding=1)
        self.pool1  = nn.MaxPool2d(2,2)

        self.conv2a = nn.Conv2d(64,  128, kernel_size=3,padding=1)
        self.conv2b = nn.Conv2d(128, 128, kernel_size=3,padding=1)
        self.pool2  = nn.MaxPool2d(2,2)

        self.conv3a = nn.Conv2d(128, 256, kernel_size=3,padding=1)
        self.conv3b = nn.Conv2d(256, 256, kernel_size=3,padding=1)
        self.pool3  = nn.MaxPool2d(2,2)
        
        self.conv4a = nn.Conv2d(256, 512, kernel_size=3,padding=1)
        self.pool4  = nn.MaxPool2d(2,2)

        self.fc1 = nn.Linear(512*2*2, 4096)
        self.fc2 = nn.Linear(4096,4096)
        self.fc3 = nn.Linear(4096, 10)


    def forward(self, x):

        # block 1: 32x32 --> 16x16
        x = self.conv1a(x)
        x = F.relu(x)
        x = self.conv1b(x)
        x = F.relu(x)
        x = self.pool1(x)

        #block 2: 16x16 --> 8x8
        x = self.conv2a(x)
        x = F.relu(x)
        x = self.conv2b(x)
        x = F.relu(x)
        x = self.pool2(x)

        #block 3: 8x8 --> 4x4
        x = self.conv3a(x)
        x = F.relu(x)
        x = self.conv3b(x)
        x = F.relu(x)
        x = self.pool3(x)

        #block 4: 4x4 --> 2x2
        x = self.conv4a(x)
        x = F.relu(x)
        x = self.pool4(x)

        # classifier
        x = x.view(-1, 512*2*2)
        x = F.relu(         self.fc1(x) )
        x = F.relu(         self.fc2(x) )
        x = F.log_softmax(  self.fc3(x) )
        
        return x

In [ ]:
max_epoch=100
batch_size_train=128
batch_size_evaluate=125
nb_cl=10


def get_lr_for_epoch(epoch):
    regimes = [
           #  start, end,    LR,   
            [  1,     30,   0.50000/2] ,
            [ 31,     60,   0.25000/2],
            [ 61,     80,   0.10000/2],
            [ 81,     100,  0.05000/2]
        ]

    for row in regimes:
        if epoch >= row[0] and epoch <= row[1]:
            LR=row[2]

    return LR

In [ ]:
mean=(0.5,0.5,0.5)
std =(0.25,0.25,0.25)

transform = transforms.Compose(
    [ transforms.ToTensor(),
      transforms.Normalize(mean, std)
     ])

transform_test = transforms.Compose(
     [  transforms.ToTensor(),
        transforms.Normalize(mean, std)
     ])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size_train,
                                          shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size_evaluate,
                                         shuffle=False, num_workers=4)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter=iter(trainloader)
images,labels = dataiter.next()
imshow(torchvision.utils.make_grid(images))

In [ ]:
net = vgg_convnet()
net.cuda()
print(net)
criterion = nn.NLLLoss()

In [ ]:
def nb_of_correct_pred(scores, targets):
    _, predicted = torch.max(scores, 1)
    correct = (predicted.squeeze() == targets)
    return correct.sum()


def train(epoch):

    lr = get_lr_for_epoch(epoch)
             
    total_loss = 0.0
    correct_pred=0
    nb_of_ex = 0
   
    for  data in trainloader:

        inputs, targets = data
        inputs, targets = Variable(inputs.cuda()), Variable(targets.cuda())

        net.zero_grad()
        scores = net(inputs)
        loss = criterion(scores, targets)
        loss.backward()

        total_loss += loss.data[0]
        correct_pred += nb_of_correct_pred(scores.data, targets.data)
        nb_of_ex +=  inputs.size(0)

        for param in net.parameters():           
            param.data -= lr * param.grad.data 

    return total_loss/len(trainloader), correct_pred, nb_of_ex, lr 
 


def evaluate():   
    
    total_loss = 0
    correct_pred = 0
    nb_of_ex = 0

    for  data in testloader:    

        inputs, targets = data
        inputs, targets = Variable(inputs.cuda()), Variable(targets.cuda())

        scores = net(inputs)
        loss = criterion(scores, targets)

        total_loss += loss.data[0]
        correct_pred += nb_of_correct_pred(scores.data, targets.data)
        nb_of_ex +=  inputs.size(0)

    return total_loss/len(testloader) , correct_pred, nb_of_ex

In [ ]:
start=time.time()

for epoch in range(1, max_epoch + 1):

    start_epoch=time.time()
    train_loss, train_correct, train_total ,lr = train(epoch)
    epoch_length=time.time()-start_epoch
    test_loss, test_correct, test_total = evaluate()
    elapsed=time.time()-start

    print('-',end='')

    print(' epoch {} ({:.1f}s) time={:.0f}min lr={:.2e}'.format(
            epoch, epoch_length, elapsed/60 ,lr  ), end='' )

    print(' || Train: loss={:.2f}, acc: {}/{} ({:.1f}%)'.format(
        train_loss, train_correct, train_total, 
        100. * train_correct / train_total ), end='')

    print(' || Test: loss={:.2f}, acc: {}/{} ({:.1f}%)'.format(
        test_loss, test_correct, test_total,
        100. * test_correct / test_total ))


 
